<img src="logo.png">


# Medidas de evaluación en clústers

Usamos métodos de evaluación interna cuando las etiquetas verdaderas no son conocidas de antemano. La desventaja principal de estos métodos es que no estamos siendo objetivos; es decir, no estamos comparando con nada que sepamos que es verdadero sino la capacidad del algoritmo para separar correctamente los datos.

## Cohesión

El criterio de cohesión da una medida de cuán juntos están los puntos dentro de un clúster.

Supongamos que hemos divido nuestra información en $k$ clústers $C_1,C_2,...C_k$. Tomando $I$ fijo, para cada observación $i\in C_I$ sea $$a_i=\frac{1}{|C_I|-1}\sum_{j\in C_I,\,j\neq i}d(i,j)$$ 

El valor $a_i$ es el promedio de todas las distancias desde la observación $i$ a todos los elementos del clúster al que pertenece. Por lo tanto, si $a_i$ es pequeño, entonces $i$ está muy cerca de todos los elementos de su clúster.

## Separación

El criterio de separación da una medida de cuán está separado un punto de los pertenecientes a los clústers a los cuales no pertenece.

Para cada $i\in C_I$, se define $$b_i=\min_{I\neq J}\frac{1}{|C_J|}\sum_{j\in C_J}d(i,j)$$

Entre mayor sea $b_i$, entonces $i$ está muy alejado de los otros clústers.

## Silueta

Ahora, para cada $i$, sea $$s_i=\frac{b_i-a_i}{\max\{a_i,b_i\}}\mbox{ si }|C_I|>1$$ y $s_i=0$ si $|C_I|=1$ (es decir, $C_I=\{i\}$).

Por lo tanto $-1\le s_i\le 1$. Al número $s_i$ se le llama *silueta de la observación $i$*.

Notemos que:
- $s_i$ es cercano a 1 si y solo si $a_i$ es cercano a 0, y por lo tanto $i$ está "bien clasificado".
- $s_i$ es cercano a -1 si y solo si $b_i$ es cercano a 0, y por lo tanto $i$ puede clasificarse mejor si se incluye en su clúster vecino.
- $s_i$ es cercano a 0 si y solo si $b_i$ y $a_i$ se parecen mucho, por lo cual $i$ es una observación complicada de clasificar.

Finalmente, se define la silueta total como el promedio de las siluetas de todos los clusters.

In [ ]:
import os
import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
from plotnine import *
from sklearn import preprocessing
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

os.chdir("C:\\Users\\hp master\\Documents\\SciData\\23_ML_Py\\data\\")

mi_data = pd.read_csv("datos_iris.csv")
mi_data = mi_data >> select(_.startswith("Sepal")) 

In [ ]:
(ggplot(data = mi_data) +
    geom_point(mapping=aes(x="Sepal_Length",y="Sepal_Width"))
)

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, silhouette_samples

mi_data = pd.read_csv("datos_iris.csv")
mi_data = mi_data >> select(_.startswith("Sepal")) 

escalador = preprocessing.normalize(mi_data)
mi_data_normalizado_df = pd.DataFrame(escalador, 
                                      index=mi_data.index, 
                                      columns=mi_data.columns)

k_medias = KMeans(n_clusters = 2 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan')
k_medias.fit(mi_data_normalizado_df)
Etiquetas = k_medias.labels_


In [ ]:
silhouette_score(mi_data_normalizado_df,Etiquetas)

In [ ]:
(mi_data >> mutate(siluetas = silhouette_samples(mi_data_normalizado_df,Etiquetas),
                  etiquetas = Etiquetas.astype(str)) >>
    ggplot() +
        geom_point(mapping=aes(x="Sepal_Length",y="Sepal_Width",color = "siluetas",shape="etiquetas"))
)

In [ ]:
def constructor_clusters(data,k):
    escalador = preprocessing.normalize(data)
    mi_data_normalizado_df = pd.DataFrame(escalador, 
                                      index=data.index, 
                                      columns=data.columns)

    k_medias = KMeans(n_clusters = k ,init='k-means++')
    k_medias.fit(mi_data_normalizado_df)
    Etiquetas = k_medias.labels_
    silueta = silhouette_score(mi_data_normalizado_df,Etiquetas)
    cal_har = calinski_harabasz_score(mi_data_normalizado_df,Etiquetas)
    
    return k, Etiquetas, silueta, cal_har 

In [ ]:
constructor_clusters(mi_data,4)

In [ ]:
modelos_kmedias = [constructor_clusters(mi_data,k) for k in range(2,10)]

In [ ]:
resultados = pd.DataFrame([(x[0],x[2],x[3]) for x in modelos_kmedias],
             columns = ["k","silueta","calinski_harabasz"])

In [ ]:
(ggplot(data = resultados) +
    geom_point(mapping = aes(x="k",y="silueta"),color = "red") +
    geom_line(mapping = aes(x="k",y="silueta"),color = "red") 
)

In [ ]:
modelos = KMeans()

visualizer = KElbowVisualizer(modelos, k=(2,10),metric = "silhouette")
visualizer.fit(mi_data_normalizado_df)
visualizer.show()

In [ ]:
modelos = KMeans()

visualizer = KElbowVisualizer(modelos, k=(2,10),metric = "calinski_harabasz")
visualizer.fit(mi_data_normalizado_df)
visualizer.show()